In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import math
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from datetime import date, datetime
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
plt.style.use("seaborn")
sns.set(style="white")

## Perfil de peliculas y usuarios.

### Perfil de las peliculas segun su genero 

In [2]:
df_genres = pd.read_csv("./data/movie_genres.csv")
df_movies = pd.read_csv("./data/movies.csv")
df_ratings = pd.read_csv("./data/ratings_train.csv")
df_ratings_test = pd.read_csv("./data/ratings_test.csv")

In [3]:
# Creo una matriz donde se agrego un flag "1" en cada genero del que la pelicula pertenece.
# Perfil de la pelicula sobre el Genero
for g in df_genres["genre"].unique():
    mids = df_genres.loc[df_genres["genre"] == g, "movieID"].unique()
    df_movies["Genre {}".format(g)] = df_movies["id"].isin(mids).astype("int16")
df_movies.head()
# Podria ponderar por la cantidad de generos que tiene, si tiene 4 generos cada uno seria 0.25

,id,title,year,imdbID,rtID,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,Genre Fantasy,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,1,Toy story,1995,114709,toy_story,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,113497,1068044-jumanji,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpy Old Men,1993,107050,grumpy_old_men,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale,1995,114885,waiting_to_exhale,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,113041,father_of_the_bride_part_ii,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Calculo para todos los usuarios

In [4]:
df_movies.head()

,id,title,year,imdbID,rtID,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,Genre Fantasy,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,1,Toy story,1995,114709,toy_story,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,113497,1068044-jumanji,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpy Old Men,1993,107050,grumpy_old_men,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale,1995,114885,waiting_to_exhale,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,113041,father_of_the_bride_part_ii,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def perfil_por_usuario(USUARIO):

    # Voy a crea el perfil de un usuario
    df_ratings_generic = df_ratings[df_ratings["userID"] == USUARIO]
    df_ratings_generic.sort_values("rating").head()
    # Ahora lo que busco es condensar sus gustos de pelicula por los generos.
    # Mergeo al rating de un usuario el perfil de cada pelicula
    df_ratings_generic = df_ratings[df_ratings["userID"] == USUARIO].merge(df_movies, left_on="movieID", right_on="id")
    df_ratings_generic.sort_values("rating").head()

    # Perfil del USER 29202, la idea es generar el perfil para los usuarios.
    # Creo PERFIL como diccionario
    PERFIL ={}
    for c in df_ratings_generic.columns:
        # Me quedo solo con las columnas que empiezan con GENRE 
        if  c.startswith("userID"):
           PERFIL[c] = [USUARIO] 
        #if not c.startswith(("Genre", "userID")):
        if not c.startswith("Genre"):
            continue
        PERFIL[c] = [df_ratings_generic.loc[df_ratings_generic[c]==1, "rating"].mean() / 5.0]


    #print(PERFIL)
    return PERFIL

In [6]:
IDs = ['29202','75','62354']
df_addoc = df_ratings.loc[df_ratings["userID"].isin(IDs)]
df_addoc = df_ratings
#df_perfil = pd.DataFrame.from_dict(PERFIL).fillna(0)
#df_perfil.insert(0,'userID',USUARIO)
df_full_profile = pd.DataFrame()
for g in tqdm_notebook(df_addoc["userID"].unique()):
    #PERFIl_FULL = perfil_por_usuario(g)
    df_full_profile = df_full_profile.append(pd.DataFrame.from_dict(perfil_por_usuario(g)).fillna(0), ignore_index=True)
    #print(df_full_profile.tail(1))


In [7]:
df_full_profile

,userID,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,Genre Fantasy,Genre Romance,Genre Drama,Genre Action,Genre Crime,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,51198,0.798000,0.788889,0.766667,0.779000,0.796667,0.810811,0.833028,0.783824,0.865116,...,0.728571,0.804762,0.784211,0.9,0.883333,0.816667,0.840000,0.800000,0.733333,0.0
1,29202,0.503182,0.447059,0.433333,0.443613,0.492742,0.536408,0.566725,0.512500,0.534911,...,0.464516,0.548750,0.509357,0.6,0.653333,0.575556,0.504545,0.442857,0.556522,0.0
2,32283,0.694545,0.806452,0.758333,0.694751,0.694805,0.723832,0.739817,0.687004,0.728571,...,0.662162,0.742857,0.656566,0.8,0.828571,0.721212,0.700000,0.800000,0.769565,0.0
3,36495,0.677551,0.763636,0.742857,0.764198,0.631579,0.740541,0.755914,0.673034,0.767241,...,0.800000,0.725000,0.640000,0.0,0.500000,0.762500,0.800000,1.000000,0.600000,0.0
4,28350,0.666667,0.705769,0.658915,0.632463,0.656180,0.670940,0.669822,0.653191,0.635714,...,0.429412,0.615000,0.597561,0.8,0.000000,0.684211,0.641667,0.700000,0.635294,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2108,61985,0.742424,0.746154,0.733333,0.709677,0.738095,0.731579,0.761538,0.749457,0.735849,...,0.738095,0.816216,0.741509,0.9,0.800000,0.768421,0.685714,0.750000,0.725000,0.0
2109,62042,0.693333,0.690000,0.693333,0.601818,0.685714,0.659375,0.763265,0.583333,0.671429,...,0.566667,0.700000,0.516667,0.0,0.000000,0.850000,0.638462,0.000000,0.800000,0.0
2110,62354,0.821429,0.820000,0.840000,0.726316,0.888889,0.768750,0.809677,0.827273,0.900000,...,0.660000,0.700000,0.840000,0.0,0.000000,0.750000,0.733333,1.000000,0.700000,0.0
2111,62394,0.619178,0.628571,0.561111,0.557534,0.613889,0.611667,0.684817,0.607477,0.708696,...,0.631373,0.664706,0.516364,0.9,0.500000,0.737037,0.650000,0.733333,0.607692,0.0


## En df_full_profile tengo el perfil de todos los usuarios y cada linea representa segun lo que califico un usuario que generos estan mas cerca o mas lejos de sus preferencia.
## Ahora lo que busco es cuales son las peliculas que estan mas cerca de casa usuario usando la Similitud Coseno (1 es super similar)

In [8]:
df_movies_adDoc = df_movies

In [9]:
# Dado un perfil de usuario, veo que pelicula es la mas afin a sus votaciones.

def pelicula_afin_a_perfil(userID, B, list_profile):
    columnas_genero = [c for c in df_movies_adDoc.columns if c.startswith("Genre")]
    
    similarityC_base=0
    movieID = 0
    #for index, row in tqdm_notebook(df_movies_adDoc.iterrows()):
    for index, row in (df_movies_adDoc.iterrows()):    
        movie = df_movies_adDoc.query("id == {}".format(row["id"]))
        A = movie[columnas_genero].values[0]
        similarityC = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
        #print(userID, row["id"], similarityC)
        if (similarityC_base < similarityC):
            similarityC_base = similarityC
            movieID = row["id"]
        #RANKING[index] = [row["id"], similarityC_base]
           
    user_row = dict()
    user_row["userID"] = userID
    user_row["movieID"] = movieID
    user_row["similarityC"] = similarityC_base
    list_profile.append(user_row)
    return list_profile

In [10]:
df_full_profileCut = df_full_profile[0:1]

In [11]:
profile_movieSelection = []
COUNT=1
for g in tqdm_notebook(df_full_profileCut["userID"].unique()):
    B=df_full_profileCut.iloc[COUNT-1:COUNT, 1:].values[0]
    pelicula_afin_a_perfil(g,B,profile_movieSelection)
    COUNT=COUNT+1


In [12]:
#df_profile_movieSelection
df_profile_movieSelection = pd.DataFrame(profile_movieSelection)
df_profile_movieSelection.head()

,userID,movieID,similarityC
0,51198,51709,0.634122


### Con esto obtuve una pelicula para recomendarle a un usuario, basado en las peliculas que ya puntuo.

### a) Poner el promedio de rating de cada pelicula.


In [13]:
df_ratings_addoc = df_ratings

In [197]:
#df_ratings_addoc["extra"] = df_ratings_addoc.loc[df_ratings_addoc['movieID'] == 141, 'rating'].sum()
#df_ratings_addoc["extra"] = df_ratings_addoc.query("movieID == 141")["rating"].sum()
#df_ratings_addoc["extra"] = df_ratings_addoc.groupby("movieID")["rating"].sum()[141]
#df_ratings_addoc["movie_avg"] = 0 

#for index, row in tqdm_notebook(df_ratings_addoc.iterrows()):
    #movieID = row["movieID"]
    #movie_avg = df_ratings_addoc.groupby("movieID")["rating"].mean()[movieID]
    #df_ratings_addoc.loc[index, 'movie_avg'] = movie_avg

In [14]:
df_ratings_addoc.head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,1,51198,608,5.0,17,9,1997,23,40,20
1,2,51198,141,4.0,17,9,1997,23,41,58
2,3,51198,1073,4.0,17,9,1997,23,41,6
3,4,51198,356,4.0,17,9,1997,23,43,44
4,5,51198,1198,5.0,17,9,1997,23,44,35


In [15]:
movies_avg = pd.DataFrame()
movies_avg = df_ratings_addoc.groupby("movieID")["rating"].mean().reset_index()
movies_avg.columns=["movieID", "movie_avg"]
movies_avg

,movieID,movie_avg
0,1,3.732032
1,2,2.977077
2,3,2.854348
3,4,2.487179
4,5,2.774038
...,...,...
10041,65088,3.500000
10042,65091,4.000000
10043,65126,3.500000
10044,65130,2.500000


In [16]:
mediaGeneral = movies_avg["movie_avg"].mean()
df_ratings_avg = df_ratings_addoc.merge(movies_avg, on="movieID", how="left")
df_ratings_avg = df_ratings_avg.fillna(mediaGeneral)
df_ratings_test_avg = df_ratings_test.merge(movies_avg, on="movieID", how="left")
df_ratings_test_avg= df_ratings_test_avg.fillna(mediaGeneral)

In [17]:
print("train: ",len(df_ratings_avg),len(df_ratings_addoc))
print("test: ",len(df_ratings_test_avg),len(df_ratings_test))

train:  770089 770089
test:  85509 85509


## Regresion Lineal inicial: Train, split de df y calculo de MSE

In [129]:
df_train = df_ratings_avg
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
full_x = df_train.drop(['rating', 'ID'], axis=1)
full_y = df_train[['rating']]
X_train, X_test, y_train, y_test = train_test_split(full_x, full_y, test_size=0.3, random_state=0)
print("train: ",len(X_train),len(y_train))
print("test:  ",len(X_test),len(y_test))
# Creo regresion lineal
model_avg = LinearRegression()
# Entreno el modelo usando el conjunto de TRAIN
model_avg.fit(X_train, y_train)
# Hago predicciones usando el TEST 
prediction_avg = model_avg.predict(X_test)
#ECM
print('Mean squared error: %.4f' % mean_squared_error(y_test, prediction_avg))


train:  539062 539062
test:   231027 231027
Mean squared error: 0.7585


## Regresion lineal: Test y escritura submit

In [82]:
df_x = df_ratings_test_avg.drop(['rating', 'ID'], axis=1)
modelFull_avg = LinearRegression()
# Entreno el modelo usando el conjunto de TRAIN
modelFull_avg.fit(X_train, y_train)

# Hago predicciones usando el TEST 
predFull_avg = modelFull_avg.predict(df_x)
print(len(df_ratings_test_avg.ID),len(predFull_avg))

85509 85509


In [83]:
def write_submit(df_test,fileName):
    submit = pd.DataFrame({'ID':df_test.ID})
    submit['rating'] = pd.DataFrame(np.around(predFull, 2))
    submit.head()
    len(submit)
    submit.to_csv("./results/"+fileName+".csv", index=False)

In [84]:
write_submit(df_ratings_test_avg, "ignacio_submit2")

In [85]:
df_ratings_test_avg.head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second,movie_avg
0,9,29202,1608,3.213463,18,9,1997,0,53,58,3.055254
1,18,32283,1485,3.213463,18,9,1997,15,45,18,3.070827
2,34,32283,30,3.213463,18,9,1997,16,36,38,3.777778
3,39,32283,1545,3.213463,18,9,1997,16,47,39,4.000000
4,50,51198,1258,3.213463,20,9,1997,0,11,21,3.964068


# *******************************************************************************************************
## LeaderBoard: 0.88102

### b) Poner el promedio de rating por mes para cada mes.

In [131]:
df_ratings_avg.head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second,movie_avg
0,1,51198,608,5.0,17,9,1997,23,40,20,4.020387
1,2,51198,141,4.0,17,9,1997,23,41,58,3.406467
2,3,51198,1073,4.0,17,9,1997,23,41,6,3.689828
3,4,51198,356,4.0,17,9,1997,23,43,44,3.931526
4,5,51198,1198,5.0,17,9,1997,23,44,35,4.136558


In [132]:
day_avg = pd.DataFrame()
day_avg = df_ratings_avg.groupby(["date_day","date_month", "date_year"])["rating"].mean().reset_index()
day_avg.columns=[ "date_day","date_month", "date_year", "day_avg"]
day_avg

,date_day,date_month,date_year,day_avg
0,1,1,1998,3.750000
1,1,1,1999,3.500000
2,1,1,2000,3.666667
3,1,1,2001,3.573333
4,1,1,2002,3.222973
...,...,...,...,...
3691,31,12,2004,3.658363
3692,31,12,2005,3.644410
3693,31,12,2006,3.040541
3694,31,12,2007,3.349217


In [133]:
mediaGeneral_day_avg = month_avg["day_avg"].mean()
df_ratings_avg_month = df_ratings_avg.merge(day_avg, left_on=["date_day", "date_month","date_year"], right_on = ["date_day", "date_month","date_year"], how="left")
df_ratings_avg_month = df_ratings_avg_month.fillna(mediaGeneral_day_avg)
df_ratings_test_avg_month = df_ratings_test_avg.merge(day_avg, left_on=["date_day", "date_month","date_year"], right_on = ["date_day","date_month","date_year"], how="left")
df_ratings_test_avg_month = df_ratings_test_avg_month.fillna(mediaGeneral_day_avg)

In [134]:
print("train: ",len(df_ratings_avg),len(df_ratings_avg_month))
print("test: ",len(df_ratings_test_avg),len(df_ratings_test_avg_month))

train:  770089 770089
test:  85509 85509


In [135]:
    df_train = df_ratings_avg_month
    full_x = df_train.drop(['rating', 'ID'], axis=1)
    full_y = df_train[['rating']]
    X_train, X_test, y_train, y_test = train_test_split(full_x, full_y, test_size=0.3, random_state=0)
    print("train: ",len(X_train),len(y_train))
    print("test:  ",len(X_test),len(y_test))
    # Creo regresion lineal
    model_avg2 = LinearRegression()
    # Entreno el modelo usando el conjunto de TRAIN
    model_avg2.fit(X_train, y_train)
    # Hago predicciones usando el TEST 
    prediction_avg2 = model_avg2.predict(X_test)
    #ECM
    print('Mean squared error: %.4f' % mean_squared_error(y_test, prediction_avg2))

train:  539062 539062
test:   231027 231027
Mean squared error: 0.7197


In [136]:
df_x = df_ratings_test_avg_month.drop(['rating', 'ID'], axis=1)
modelFull_avg2 = LinearRegression()
# Entreno el modelo usando el conjunto de TRAIN
modelFull_avg2.fit(X_train, y_train)

# Hago predicciones usando el TEST 
predFull_avg2 = modelFull_avg2.predict(df_x)
print(len(df_ratings_test_avg.ID),len(predFull_avg2))

85509 85509


In [137]:
write_submit(df_ratings_test_avg_month, "ignacio_submit3")

## LeaderBoard: 0.88102